In [1]:
pip install gpxpy pandas folium pywhatkit


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import gpxpy
import folium
import webbrowser
from math import radians, cos, sin, sqrt, atan2
from datetime import datetime, timedelta

# Function to parse GPX file
def parse_gpx(file_path):
    with open(file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    return gpx

# Function to extract data from GPX file into a DataFrame
def gpx_to_dataframe(gpx):
    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append([point.latitude, point.longitude, point.elevation, point.time])
    df = pd.DataFrame(data, columns=['latitude', 'longitude', 'elevation', 'time'])
    return df

# Haversine formula to calculate distance between two points on the Earth
def haversine(point1, point2):
    lat1, lon1 = point1
    lat2, lon2 = point2
    R = 6371.0  # Radius of the Earth in kilometers

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

# Function to add labels every 5 kilometers with distance and running time labels in a squared box with 50% transparency
def add_distance_labels(df, map_obj, color):
    total_distance = 0
    prev_point = None
    km_marker = 5
    for _, row in df.iterrows():
        current_point = (row['latitude'], row['longitude'])
        if prev_point is not None:
            distance = haversine(prev_point, current_point)
            total_distance += distance
            if total_distance >= km_marker:
                running_time = row['running_time'].strftime("%H:%M")
                folium.Marker(
                    location=current_point,
                    icon=folium.DivIcon(
                        html=f'<div style="font-size: 12pt; color: {color}; background-color: rgba(255, 255, 255, 0.9); border: 1px solid {color}; padding: 5px; width: 50px;">{km_marker} km<br>{running_time}</div>'
                    )
                ).add_to(map_obj)
                km_marker += 5
        prev_point = current_point

# Function to calculate running time for each row based on speed and start time
def calculate_running_time(df, speed_kmh, start_time):
    total_distance = 0
    prev_point = None
    running_times = []
    
    for _, row in df.iterrows():
        current_point = (row['latitude'], row['longitude'])
        if prev_point is not None:
            distance = haversine(prev_point, current_point)
            total_distance += distance
            time_elapsed_hours = total_distance / speed_kmh
            running_time = start_time + timedelta(hours=time_elapsed_hours)
        else:
            running_time = start_time
        
        running_times.append(running_time)
        prev_point = current_point
    
    return running_times

# Parse the two GPX files
gpx1 = parse_gpx(r"C:\Users\ScholtenR\Downloads\file1.gpx")
gpx2 = parse_gpx(r"C:\Users\ScholtenR\Downloads\file2.gpx")

# Convert GPX files to DataFrames
df1 = gpx_to_dataframe(gpx1)
df2 = gpx_to_dataframe(gpx2)

# Calculate running time for each row and add it as a new column in each dataframe
start_time_df1 = datetime.strptime("12:00", "%H:%M")
start_time_df2 = datetime.strptime("10:00", "%H:%M")

df1['running_time'] = calculate_running_time(df1, 10, start_time_df1)
df2['running_time'] = calculate_running_time(df2, 12, start_time_df2)

# Create a map centered around the average latitude and longitude of both dataframes
map_center = [(df1['latitude'].mean() + df2['latitude'].mean()) / 2, (df1['longitude'].mean() + df2['longitude'].mean()) / 2]
mymap = folium.Map(location=map_center, zoom_start=13)

# Add the GPX data to the map with different colors and add distance labels every 5 kilometers with labels
folium.PolyLine(df1[['latitude', 'longitude']].values.tolist(), color='blue', label='Half Marathon').add_to(mymap)
folium.PolyLine(df2[['latitude', 'longitude']].values.tolist(), color='red', label='Marathon').add_to(mymap)

add_distance_labels(df1, mymap, 'blue')
add_distance_labels(df2, mymap, 'red')

# Add legend to the map
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 300px; height: 90px; 
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
     <i class="fa fa-map-marker fa-2x" style="color:blue"></i> Half Marathon, start om 12:00, 10 km/u<br>
     <i class="fa fa-map-marker fa-2x" style="color:red"></i> Marathon, start om 10:00, 12 km/u<br>
     </div>
     '''
mymap.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
map_file = "combined_gpx_map.html"
mymap.save(map_file)

# Open the map in a browser
webbrowser.open(map_file)

print("Map has been saved to combined_gpx_map.html and opened in your default browser.")
print("Running times for GPX1 (10 km/h):")
print(df1[['latitude', 'longitude', 'running_time']])
print("Running times for GPX2 (12 km/h):")
print(df2[['latitude', 'longitude', 'running_time']])


Map has been saved to combined_gpx_map.html and opened in your default browser.
Running times for GPX1 (10 km/h):
     latitude  longitude               running_time
0   51.445835   5.476921 1900-01-01 12:00:00.000000
1   51.448109   5.477350 1900-01-01 12:01:31.634347
2   51.450168   5.477607 1900-01-01 12:02:54.321700
3   51.451800   5.477908 1900-01-01 12:04:00.055235
4   51.452976   5.477951 1900-01-01 12:04:47.171165
..        ...        ...                        ...
75  51.435965   5.482800 1900-01-01 14:03:26.718129
76  51.434413   5.483744 1900-01-01 14:04:33.151158
77  51.432513   5.484774 1900-01-01 14:05:53.417809
78  51.432353   5.483916 1900-01-01 14:06:15.781113
79  51.432968   5.482628 1900-01-01 14:06:56.267280

[80 rows x 3 columns]
Running times for GPX2 (12 km/h):
      latitude  longitude               running_time
0    51.452469   5.477476 1900-01-01 10:00:00.000000
1    51.452469   5.477475 1900-01-01 10:00:00.020788
2    51.454216   5.477207 1900-01-01 10:00:58.

In [3]:
df2


,latitude,longitude,elevation,time,running_time
0,51.452469,5.477476,21.0,2024-10-09 00:00:00+00:00,1900-01-01 10:00:00.000000
1,51.452469,5.477475,21.0,2024-10-09 00:00:00+00:00,1900-01-01 10:00:00.020788
2,51.454216,5.477207,23.0,2024-10-09 00:01:10+00:00,1900-01-01 10:00:58.563724
3,51.454349,5.477245,23.0,2024-10-09 00:01:16+00:00,1900-01-01 10:01:03.070171
4,51.455885,5.476980,20.0,2024-10-09 00:02:18+00:00,1900-01-01 10:01:54.604037
...,...,...,...,...,...
816,51.438083,5.482255,22.0,2024-10-09 04:19:06+00:00,1900-01-01 13:35:40.288713
817,51.438203,5.482124,22.0,2024-10-09 04:19:12+00:00,1900-01-01 13:35:45.130678
818,51.438238,5.482339,22.0,2024-10-09 04:19:17+00:00,1900-01-01 13:35:49.751387
819,51.438266,5.482322,23.0,2024-10-09 04:19:18+00:00,1900-01-01 13:35:50.750081
